In [1]:
import pandas as pd
import geopandas as gpd
import shapely
import numpy as np


In [2]:
CA_domestic=gpd.read_file('CA_DomesticWls_HH.shp')


In [4]:
zip_gdf = gpd.read_file("../AGU_data/census_zipcode/central_valley_zip.shp")[['Zipcode','geometry']]
zip_gdf=zip_gdf.to_crs('4326')

In [5]:
CA_domestic

,MTRS,All_Wells,Dom_Wells,TRatio,Dom_HH,SLO_Co,Area,geometry
0,BAY/DELTA,0.0,0.0,0.0,0.800971,0,1.193196e+09,"POLYGON ((-200381.135 1686130.178, -200374.046..."
1,H01N01E01,0.0,0.0,0.4,0.000000,0,2.567345e+06,"POLYGON ((-336555.822 1949337.348, -336578.656..."
2,H01N01E02,0.0,0.0,0.4,0.000000,0,2.597895e+06,"POLYGON ((-338118.323 1949150.786, -338116.562..."
3,H01N01E03,0.0,0.0,0.4,0.000000,0,2.548868e+06,"POLYGON ((-339746.017 1949450.786, -339751.374..."
4,H01N01E04,0.0,0.0,0.4,0.000000,0,2.771691e+06,"POLYGON ((-341279.907 1950294.001, -341433.249..."
...,...,...,...,...,...,...,...,...
158660,S30N02E33,0.0,0.0,0.0,0.000000,0,2.425789e+04,"POLYGON ((285503.375 1519357.001, 285309.593 1..."
158661,S31N01E30,0.0,0.0,0.0,0.000000,0,4.506578e+05,"POLYGON ((273191.793 1530367.731, 272264.656 1..."
158662,S31N01E31,0.0,0.0,0.0,0.000000,0,2.356735e+06,"POLYGON ((273082.500 1528749.749, 272318.594 1..."
158663,S31N01E32,0.0,0.0,0.0,0.000000,0,6.083519e+05,"POLYGON ((274972.532 1528798.500, 273852.438 1..."


In [7]:
# Ensure all GeoDataFrames are in CRS 4326
CA_domestic = CA_domestic.to_crs(epsg=4326)
# water_systems = water_systems.to_crs(epsg=4326)
zip_gdf = zip_gdf.to_crs(epsg=4326)

geometries= CA_domestic.geometry
centroids=geometries.apply(lambda x:x.centroid)
CA_domestic['centroid']=centroids

CA_domestic.head()

,MTRS,All_Wells,Dom_Wells,TRatio,Dom_HH,SLO_Co,Area,geometry,centroid,Longitude,Latitude
0,BAY/DELTA,0.0,0.0,0.0,0.800971,0,1.193196e+09,"POLYGON ((-122.30978 38.18620, -122.30967 38.1...",POINT (-122.26502 37.85705),-122.265021,37.857054
1,H01N01E01,0.0,0.0,0.4,0.000000,0,2.567345e+06,"POLYGON ((-124.00584 40.49330, -124.00591 40.4...",POINT (-124.01534 40.49707),-124.015342,40.497069
2,H01N01E02,0.0,0.0,0.4,0.000000,0,2.597895e+06,"POLYGON ((-124.02433 40.49105, -124.02424 40.4...",POINT (-124.03430 40.49707),-124.034302,40.497070
3,H01N01E03,0.0,0.0,0.4,0.000000,0,2.548868e+06,"POLYGON ((-124.04383 40.49311, -124.04371 40.4...",POINT (-124.05345 40.49700),-124.053448,40.496996
4,H01N01E04,0.0,0.0,0.4,0.000000,0,2.771691e+06,"POLYGON ((-124.06249 40.50004, -124.06375 40.4...",POINT (-124.07292 40.49748),-124.072920,40.497483


In [8]:
well_locations=gpd.GeoDataFrame(geometry=CA_domestic['centroid'])
well_locations['Dom_HH']=CA_domestic['Dom_HH']
well_locations

,geometry,Dom_HH
0,POINT (-122.26502 37.85705),0.800971
1,POINT (-124.01534 40.49707),0.000000
2,POINT (-124.03430 40.49707),0.000000
3,POINT (-124.05345 40.49700),0.000000
4,POINT (-124.07292 40.49748),0.000000
...,...,...
158660,POINT (-116.77622 36.67715),0.000000
158661,POINT (-116.91654 36.78124),0.000000
158662,POINT (-116.91199 36.77087),0.000000
158663,POINT (-116.89849 36.76712),0.000000


In [9]:
joined = gpd.sjoin(well_locations, zip_gdf, how="left", predicate="within")
joined=joined.dropna(subset=['Zipcode'])
joined

,geometry,Dom_HH,index_right,Zipcode
6941,POINT (-121.81456 37.96108),0.667118,340.0,94531
6944,POINT (-121.86954 37.96120),0.750149,243.0,94565
6946,POINT (-121.90591 37.96148),2.483761,208.0,94521
6959,POINT (-121.90461 37.91782),4.597640,286.0,94517
6960,POINT (-121.88621 37.91786),0.000000,286.0,94517
...,...,...,...,...
146282,POINT (-119.32276 35.08352),0.000000,17.0,93311
146287,POINT (-119.42753 35.09253),0.000000,348.0,93268
146288,POINT (-119.44574 35.09258),0.000000,348.0,93268
146293,POINT (-119.44614 35.08411),0.000000,348.0,93268


In [10]:
joined.to_csv('wells_zipcodes_DomHH.csv')